In [35]:
label_data = '/home/hao/Project/MERTools/MER2024/mer2024-dataset-process/label-6way.npz'
import numpy as np

label = np.load(label_data, allow_pickle=True)
print(label.files)

test1_corpus = label['test1_corpus'].item()  # Convert to dictionary
print(type(test1_corpus))  # Should print <class 'dict'>

# Now you can inspect the keys and values of the dictionary
#print(test1_corpus.keys())  # Print all the keys in the dictionary
a=0
for key, value in test1_corpus.items():
    #print(f"Key: {key}, Value type: {type(value)}, Value shape: {np.shape(value)}")
    pass
    a+=1
print(a)
#print(len(test1_corpus.item()))

['train_corpus', 'test1_corpus']
<class 'dict'>
20000


In [10]:
#data_path = '/home/hao/Project/MERTools/MER2024/saved-unimodal/result/cv_features:chinese-hubert-large-UTT_dataset:MER2024_model:attention+utt+None_f1:0.7195_acc:0.7215_1718880970.9861982.npz'
data_path='/home/hao/Project/MERTools/MER2024/saved-unimodal/result/test1_features:chinese-hubert-large-UTT_dataset:MER2024_model:attention+utt+None_f1:0.7362_acc:0.5825_1719147356.599465.npz'
import numpy as np
data = np.load(data_path, allow_pickle=True)

print(data.files)  # 打印所有的 key 值
# 假设 'args' 是以 Namespace 对象保存的，使用 vars() 函数转换为字典
args_dict = vars(data['args'].item())  # 先使用 .item() 将 numpy 对象转换为原生 Python 对象，然后使用 vars() 转换 Namespace 为字典

# 打印所有参数
for key, value in args_dict.items():
    print(f"{key}: {value}")

['emo_probs', 'args']
dataset: MER2024
train_dataset: None
test_dataset: None
train_snr: None
test_snr: None
fusion_topn: None
fusion_modality: AVT
save_root: ./saved-unimodal
debug: False
savemodel: False
save_iters: 100000000.0
audio_feature: chinese-hubert-large-UTT
text_feature: chinese-hubert-large-UTT
video_feature: chinese-hubert-large-UTT
feat_type: utt
feat_scale: 1
e2e_name: None
e2e_dim: None
n_classes: None
hyper_path: None
model: attention
lr: 0.001
lr_adjust: case1
l2: 1e-05
batch_size: 32
num_workers: 0
epochs: 100
print_iters: 100000000.0
gpu: 1
hidden_dim: 128
dropout: 0.2
grad_clip: -1.0
output_dim1: 6
output_dim2: 0
metric_name: emo
snr: None
audio_dim: 1024
text_dim: 1024
video_dim: 1024
duration: 2574.6658017635345


In [14]:
print(data['emo_probs'])

[[-0.5401095  -1.1594989  -2.4768634   5.0378823   0.5163256  -2.6562953 ]
 [ 3.4656467  -2.210908    0.75659287 -0.9314119   0.40232545 -2.6206052 ]
 [ 3.3874233  -1.0452689   1.2425289  -1.1382792  -1.1326225  -3.0681567 ]
 ...
 [-1.2012459   3.7484183  -0.02676558 -1.0035951  -0.8062688  -3.0043454 ]
 [ 4.244792   -0.8595301  -0.83916265 -0.14125796 -0.8671662  -3.6432579 ]
 [ 1.5819685  -1.4470174  -1.5424184   2.1403646   0.41163498 -2.3904977 ]]


In [3]:


import torch
import torch.nn.functional as F
import soundfile as sf

from transformers import (
    Wav2Vec2FeatureExtractor,
    HubertModel,
)


model_path="tools/transformers/chinese-hubert-large"
wav_path="/home/hao/Project/MERTools/MER2024/9426.wav"

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)
model = HubertModel.from_pretrained(model_path)



device = "cuda" if torch.cuda.is_available() else "cpu"

model = model.to(device)
model = model.half()
model.eval()


Some weights of the model checkpoint at tools/transformers/chinese-hubert-large were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at tools/transformers/chinese-hubert-large and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to

HubertModel(
  (feature_extractor): HubertFeatureEncoder(
    (conv_layers): ModuleList(
      (0): HubertLayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x HubertLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x HubertLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): HubertFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, 

In [7]:

wav, sr = sf.read(wav_path)
input_values = feature_extractor(wav, return_tensors="pt").input_values
input_values = input_values.half()
input_values = input_values.to(device)

import torch.nn as nn

# 假设 HubertModel 的 last_hidden_state 维度是 hidden_size
hidden_size = model.config.hidden_size  # 获取模型的隐藏层大小
desired_feature_size = 4096  # 目标特征维度

# 定义一个全连接层来调整维度
# 在定义全连接层后，将其转换为半精度
adjust_layer = nn.Linear(hidden_size, desired_feature_size).to(device).half()

# 将 HubertModel 的输出通过这个全连接层
with torch.no_grad():
    outputs = model(input_values)
    last_hidden_state = outputs.last_hidden_state
    # 我们只对最后一个时间步的输出感兴趣
    last_hidden_state = last_hidden_state[:, -1, :]
    adjusted_features = adjust_layer(last_hidden_state)  # 调整维度至4096




It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [8]:
print(adjusted_features.shape)  # torch.Size([1, 4096])

torch.Size([1, 4096])


In [1]:
new_candidate_path = '/data/public_datasets/MER_2024/new_release/candidate_20000.csv'

import pandas as pd

candidate = pd.read_csv(new_candidate_path)
candidate_names = set(candidate['name'].tolist())


In [2]:
import os
import shutil

unlabeled_audio_path = '/data/public_datasets/MER_2024/new_release/unlabeled-with-test2noise-audio/mer2024_unlabel_audio'
labeled_audio_path = '/data/public_datasets/MER_2024/audio-labeled_16k'

unlabeled_audio_names = os.listdir(unlabeled_audio_path)
unlabeled_audio_names = [item.rsplit('.')[0] for item in unlabeled_audio_names]
choose_20000 = [name for name in unlabeled_audio_names if name in candidate_names]
assert len(choose_20000) == 20000

labeled_audio_name = os.listdir(labeled_audio_path)

save_root = '/data/public_datasets/MER_2024/mer2024-dataset-process/audio'
if not os.path.exists(save_root):
    os.makedirs(save_root)

print(f'we have {len(labeled_audio_name)} labeled audio and {len(choose_20000)} unlabeled audio')

# Copy labeled audio
for audio_name in labeled_audio_name:
    source_path = os.path.join(labeled_audio_path, audio_name)
    destination_path = os.path.join(save_root, audio_name)
    if not os.path.exists(destination_path):  # Check if the file already exists
        shutil.copyfile(source_path, destination_path)

# Copy unlabeled audio
for audio_name in choose_20000:
    source_path = os.path.join(unlabeled_audio_path, audio_name + '.wav')
    destination_path = os.path.join(save_root, audio_name + '.wav')
    if not os.path.exists(destination_path):  # Check if the file already exists
        shutil.copyfile(source_path, destination_path)

we have 5024 labeled audio and 20000 unlabeled audio


In [5]:
labeled_face_path = '/data/public_datasets/MER_2024/mer2024-dataset-process/croped_face/labeled_croped'
unlabeled_face_path = '/data/public_datasets/MER_2024/mer2024-dataset-process/croped_face/unlabeled_croped'
save_face_root = '/data/public_datasets/MER_2024/mer2024-dataset-process/croped'
# copy all labeled_face and candidate_face to save_face_root

unlabel_face_names = os.listdir(unlabeled_face_path)
label_face_names = os.listdir(labeled_face_path)

choose_20000_face = [name for name in unlabel_face_names if name in candidate_names]
assert len(choose_20000_face) == 20000

if not os.path.exists(save_face_root):
    os.makedirs(save_face_root)

print(label_face_names)
print(choose_20000_face)
print(f'we have {len(label_face_names)} labeled face and {len(choose_20000_face)} unlabeled face')

# copy labeled face
for face_name in label_face_names:
    source_path = os.path.join(labeled_face_path, face_name)
    destination_path = os.path.join(save_face_root, face_name)
    if not os.path.exists(destination_path):  # Check if the file already exists
        shutil.copyfile(source_path, destination_path)

# copy unlabeled face
for face_name in choose_20000_face:
    source_path = os.path.join(unlabeled_face_path, face_name)
    destination_path = os.path.join(save_face_root, face_name)
    if not os.path.exists(destination_path):  # Check if the file already exists
        shutil.copyfile(source_path, destination_path)



['sample_00005474', 'sample_00002576', 'sample_00001685', 'sample_00000446', 'sample_00005962', 'sample_00005160', 'samplenew_00039171', 'sample_00006255', 'sample_00001666', 'sample_00004143', 'sample_00002152', 'sample_00001153', 'sample_00001098', 'sample_00006121', 'sample_00003631', 'sample_00002932', 'sample_00005449', 'sample_00001999', 'sample_00002437', 'sample_00003385', 'sample_00000961', 'sample_00005688', 'samplenew_00056813', 'sample_00001456', 'sample_00003204', 'sample_00003931', 'sample_00001634', 'samplenew_00010034', 'sample_00006557', 'sample_00004649', 'sample_00006550', 'sample_00005210', 'sample_00003407', 'sample_00006520', 'sample_00001183', 'sample_00005169', 'samplenew_00003986', 'sample_00002012', 'sample_00005340', 'sample_00006945', 'sample_00003284', 'sample_00000888', 'sample_00004487', 'sample_00004492', 'sample_00002888', 'sample_00000434', 'sample_00004159', 'sample_00000253', 'samplenew_00040661', 'sample_00005666', 'sample_00006027', 'sample_0000652